# Supervised Baseline for Dataset Pets-2009

In [ ]:
from IPython.display import display, HTML

# Defina a largura desejada para a célula
largura_celula = '90%'

# Use a função display() para definir a largura do output
display(HTML(f"<style>.container {{ width:{largura_celula} !important; }}</style>"))

In [ ]:
!ls drive/MyDrive/Colab-Data/PhD/UpFall

models						 upfall-flow-train-single-camera-c1-seed1.npy
upfall-combine-test-single-camera-c1-seed1.npy	 upfall-flow-train-single-camera-c2-seed1.npy
upfall-combine-test-single-camera-c2-seed1.npy	 upfall-rgb-test-single-camera-c1-seed1.npy
upfall-combine-train-single-camera-c1-seed1.npy  upfall-rgb-test-single-camera-c2-seed1.npy
upfall-combine-train-single-camera-c2-seed1.npy  upfall-rgb-train-single-camera-c1-seed1.npy
upfall-flow-test-single-camera-c1-seed1.npy	 upfall-rgb-train-single-camera-c2-seed1.npy
upfall-flow-test-single-camera-c2-seed1.npy


In [ ]:
dir_feats = "drive/MyDrive/Colab-Data/PhD/UpFall/"
#view = "001"
#feat_mode = "rgb"

In [ ]:
!pip install ipython-autotime

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
!pip install lightgbm --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.4.0
    Uninstalling lightgbm-4.4.0:
      Successfully uninstalled lightgbm-4.4.0


In [ ]:
!pip install munch

In [ ]:
'''
Import all the necessary packages and modules
'''
import os, sys
import munch
import random
import joblib
import time
from collections import Counter
import numpy as np
import pandas as pd
from IPython.display import HTML, display, clear_output
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC

import sklearn
import sklearn.metrics

import lightgbm
import lightgbm as lgb

%matplotlib inline

time: 1.81 ms (started: 2024-08-10 18:10:14 +00:00)


In [ ]:
def beautify_df_lsf(mtx_mean, mtx_min, mtx_max, mtx_std):

    mtx_mean = list(mtx_mean.astype(str))
    mtx_std = list(mtx_std.astype(str))
    mtx_min = list(mtx_min.astype(str))
    mtx_max = list(mtx_max.astype(str))

    row = list()
    for j in range(7):
        cell = f'{mtx_mean[j]} ± {mtx_std[j]} ({mtx_min[j]},{mtx_max[j]})'
        row.append(cell)
    df = pd.DataFrame([row])
    df.columns = ['AUC','FPR','ACC','BACC','PREC','REC','F1']
    return df

def beautify_df_mc(mtx_mean, mtx_min, mtx_max, mtx_std):
    # Cria Matriz com as Informações
    row = list()
    for j in range(7):
        cell = f'{mtx_mean[j]} ± {mtx_std[j]} ({mtx_min[j]},{mtx_max[j]})'
        row.append(cell)
    df = pd.DataFrame([row])
    df.columns = ['AUC','FPR','ACC','BACC','PREC','REC','F1']
    return df

def prepare_mean(tmp):
  tmp = np.array(tmp)
  for i in range(tmp.shape[0]):
    for j in range(tmp.shape[1]):
      tmp[i][j] = eval(tmp[i][j].split(" ±")[0])

  _mean = np.mean(tmp.astype(np.float32),axis=0).round(2)
  _std = np.std(tmp.astype(np.float32),axis=0).round(2)
  _min = np.min(tmp.astype(np.float32),axis=0).round(2)
  _max = np.max(tmp.astype(np.float32),axis=0).round(2)

  return beautify_df_lsf(_mean,_min,_max, _std)

time: 2.5 ms (started: 2024-08-10 18:10:15 +00:00)


In [ ]:
def gera_segmentos(videos_list):
  '''
  Extrai clips a partir das bags
  '''
  clips_list = list()
  segment_labels = {}
  for index, video in enumerate(videos_list):
    video_name = video['name']
    y_fi_reshaped = videos_list[index]["y_fi"].reshape([len(video["X_i"]), 16])
    for clip_index, clip in enumerate(video["X_i"]):
      clip_frame_labels = y_fi_reshaped[clip_index]
      clip_label = videos_list[index]["y_i"]
      clips_list.append(dict(clip = clip,
                             label = clip_label,
                             frame_labels = clip_frame_labels,
                             video_name = video_name))
      if video_name not in segment_labels:
          segment_labels[video_name] = list()
      segment_labels[video_name].append(clip_label)

  print(f'Loaded data: {len(clips_list)} clips into {len(videos_list)} videos.')
  return clips_list, segment_labels

time: 1.26 ms (started: 2024-08-10 18:10:17 +00:00)


In [ ]:
def List4Xyyfl(clips_list):
        '''# Transforma o dataset de clips em um conjunto na forma X, y'''
        X, y, y_fl = list(), list(), list()
        for clip in clips_list:
            X.append(clip['clip'])
            y.append(clip['label'])
            y_fl.append(clip['frame_labels'])

        X = np.array(X)
        y = np.array(y)
        y_fl = np.array(y_fl)
        y = y.reshape([len(y), 1])
        return X, y, y_fl

time: 1.17 ms (started: 2024-08-10 18:10:17 +00:00)


In [ ]:
def transform_y_pred(y_pred):
    '''
    @param y: a vector with n values
    @return: a matrix nx16 replicating each value along the columns
    '''
    my = list()
    for yp in y_pred:
        my.append([yp]*16)
    my = np.array(my)
    return my.ravel()

time: 538 µs (started: 2024-08-10 18:10:20 +00:00)


In [ ]:
def compute_performance_metrics(y_true, y_score, y_predicted):

    #fpr_keras, tpr_keras, thresholds_keras = sklearn_metrics.roc_curve(y_true, y_score)
    cm = sklearn.metrics.confusion_matrix(y_true, y_predicted)
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true, y_predicted).ravel()

    acc = sklearn.metrics.balanced_accuracy_score(y_true, y_predicted)

    auc = sklearn.metrics.roc_auc_score(y_true, y_score)
    fpr = fp/(fp + tn)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    #print(f'TN:{tn} FP:{fp} FN:{fn} TP:{tp}')
    #print('\tNormal\tAnom')
    #print(f'Normal\t{cm[0, 0]}\t{cm[0, 1]}')
    #print(f'Anom\t{cm[1, 0]}\t{cm[1, 1]}')

    df_test_result = pd.DataFrame(data=[[
        round(auc*100,2),round(fpr*100,2), round(f1*100,2)
    ]], columns=['AUC', 'FPR', 'F1'], index=['']) # , index = [st]

    return df_test_result

time: 743 µs (started: 2024-08-10 18:10:21 +00:00)


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve

def compute_performance_metrics(y_true, y_score, y_predicted):

    all_y_true = y_true
    all_y_pred = y_score

    fpr, tpr, thresholds = roc_curve(all_y_true, all_y_pred)
    _auc = round(sklearn.metrics.auc(fpr, tpr) * 100, 2)

    cm = confusion_matrix(all_y_true, np.round(all_y_pred))
    tn, fp, fn, tp = confusion_matrix(all_y_true, np.round(all_y_pred)).ravel()
    fpr = fp/(fp + tn)
    fpr = round(fpr * 100, 2)
    _fpr = fpr

    _acc = (tp+tn)/(tp+tn+fp+fn) * 100
    _bacc = sklearn.metrics.balanced_accuracy_score(all_y_true, np.round(all_y_pred))  * 100

    _prec = tp/(tp+fp) * 100
    _rec = tp/(tp+fn) * 100
    _f1 = 2 * (_prec * _rec) / (_prec + _rec)

    # AUC,FPR,ACC,BACC,PREC,REC,F1
    return [_auc, _fpr, _acc, _bacc, _prec, _rec, _f1]

time: 923 µs (started: 2024-08-10 18:10:22 +00:00)


time: 1.73 s (started: 2024-08-10 18:10:22 +00:00)


In [ ]:
def slice_by_view(data, view):
  if view == 'rgb':
    return data[:, :1024]
  elif view == 'flow':
    return data[:, 1024:]
  elif view == 'combine':
    return data

time: 779 µs (started: 2024-08-10 18:10:25 +00:00)


In [ ]:
#def slice_by_view_two_cams(data, view):
#  if view == 'rgb':
#    return np.hstack([data[:, :1024], data[:, 2048:3072] ])
#  elif view == 'flow':
#    raise
#  elif view == 'combine':
#    return data

time: 271 µs (started: 2024-08-10 18:10:26 +00:00)


In [ ]:
def load_data(path_data):
  data = munch.munchify(np.load(path_data, allow_pickle=True))
  data_clips, data_clip_labels = gera_segmentos(videos_list = data)
  X, y, y_fl = List4Xyyfl(clips_list = data_clips)
  return X, y, y_fl

time: 759 µs (started: 2024-08-10 18:11:01 +00:00)


In [ ]:
def get_splits(feat_mode, camera):
    path_train = os.path.join(dir_feats, f"upfall-{feat_mode}-train-single-camera-{camera}-seed1.npy")
    path_test = os.path.join(dir_feats,  f"upfall-{feat_mode}-test-single-camera-{camera}-seed1.npy")
    X_train, y_train, y_fl_train =  load_data(path_train)
    y_train = (np.sum(y_fl_train, axis = 1) > True).astype(float).ravel()
    X_test, y_test, y_fl_test =  load_data(path_test)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    return X_train, y_train, y_fl_train, X_test, y_test, y_fl_test

time: 1.1 ms (started: 2024-08-10 18:12:15 +00:00)


# Training

In [ ]:
%load_ext autotime
for cA,cB in [
    ("c1","c2"),
    ]:
  print("pair", cA, cB)
  for feat_mode in ["rgb", "flow", "combine"]:

    # load train and test splits
    X_train_cA, y_train_cA, y_fl_train_cA, X_test_cA, y_test_cA, y_fl_test_cA = get_splits(feat_mode,cA)
    X_train_cB, y_train_cB, y_fl_train_cB, X_test_cB, y_test_cB, y_fl_test_cB = get_splits(feat_mode, cB)
    #X_train_cA = slice_by_view(X_train_cA, feat_mode)
    #X_test_cA = slice_by_view(X_test_cA, feat_mode)
    #X_train_cB = slice_by_view(X_train_cB, feat_mode)
    #X_test_cB = slice_by_view(X_test_cB, feat_mode)
    X_train = np.hstack([X_train_cA, X_train_cB])
    y_train = y_train_cA

    N_ESTIMATORS = 5000
    for exec_index in range(5):
      model  = lgb.LGBMClassifier(n_estimators=N_ESTIMATORS) # , random_state = 42
      model = model.fit(X_train, y_train)
      clear_output()
      np.save(os.path.join(dir_feats, "models", f"lgbm-upfall-{feat_mode}-pair-{cA}-{cB}-exec-{exec_index+1}.npy"), model)
      clear_output()
clear_output()

time: 2h 6min 44s (started: 2024-08-10 18:12:16 +00:00)


In [ ]:
!ls drive/MyDrive/Colab-Data/PhD/UpFall/

models						 upfall-flow-train-single-camera-c1-seed1.npy
upfall-combine-test-single-camera-c1-seed1.npy	 upfall-flow-train-single-camera-c2-seed1.npy
upfall-combine-test-single-camera-c2-seed1.npy	 upfall-rgb-test-single-camera-c1-seed1.npy
upfall-combine-train-single-camera-c1-seed1.npy  upfall-rgb-test-single-camera-c2-seed1.npy
upfall-combine-train-single-camera-c2-seed1.npy  upfall-rgb-train-single-camera-c1-seed1.npy
upfall-flow-test-single-camera-c1-seed1.npy	 upfall-rgb-train-single-camera-c2-seed1.npy
upfall-flow-test-single-camera-c2-seed1.npy
time: 111 ms (started: 2024-08-10 18:07:32 +00:00)


## Evaluation

In [ ]:
!pip install lightgbm --upgrade
!pip install munch

In [ ]:
import os, sys
import munch
import random
import joblib
import time
from collections import Counter
import numpy as np
import pandas as pd
from IPython.display import HTML, display, clear_output
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC

import sklearn
import sklearn.metrics

import lightgbm
import lightgbm as lgb

%matplotlib inline

time: 11.3 ms (started: 2024-08-10 21:21:32 +00:00)


In [ ]:
dfsbyfeatmodes = list()
for feat_mode in  ["rgb","flow","combine"]:
  df_views = list()
  for cA,cB in [("c1","c2"),]:
    rows_result = list()
    for exec_index in [1,2,3,4,5]:
      m = np.load(os.path.join(dir_feats, "models", f"lgbm-upfall-{feat_mode}-pair-{cA}-{cB}-exec-{exec_index}.npy"), allow_pickle=True).item()

      # load testing data
      X_train_cA, y_train_cA, y_fl_train_cA, X_test_cA, y_test_cA, y_fl_test_cA = get_splits(feat_mode, cA)
      X_train_cB, y_train_cB, y_fl_train_cB, X_test_cB, y_test_cB, y_fl_test_cB = get_splits(feat_mode, cB)
      #X_test_cA = slice_by_view(X_test_cA, feat_mode)
      #X_test_cB = slice_by_view(X_test_cB, feat_mode)
      X_test = np.hstack([X_test_cA, X_test_cB])
      y_test = y_test_cA
      y_fl_test = y_fl_test_cA

      y_probs = m.predict_proba(X_test)[:, 1]
      y_predicted = (y_probs > 0.5).astype("int32")

      evaluation_results = dict()
      evaluation_results['frame-level'] = [y_fl_test.ravel(), transform_y_pred(y_probs), transform_y_pred(y_predicted)]
      y_testr, y_probsr, y_predictedr = evaluation_results['frame-level']

      y_testr[np.isnan(y_testr)] = 0
      rows_result.append(compute_performance_metrics(y_testr, y_probsr, y_predictedr))

    # Gera Matriz para Impressão
    mtx_mean = np.round(np.array(rows_result).mean(axis=0), decimals=2).ravel()
    mtx_std = np.round(np.array(rows_result).std(axis=0), decimals=2).ravel()
    mtx_min = np.round(np.array(rows_result).min(axis=0), decimals=2).ravel()
    mtx_max = np.round(np.array(rows_result).max(axis=0), decimals=2).ravel()

    df_view = beautify_df_lsf(mtx_mean, mtx_min, mtx_max, mtx_std)
    df_view.index = [f"{cA}-{cB}"]
    df_views.append(df_view)
    clear_output()

  results_mean = prepare_mean(pd.concat(df_views))
  results_mean.index = ["Mean"]
  dfsbyfeatmodes.append(pd.concat([
      pd.concat(df_views), results_mean
  ]))

time: 37 s (started: 2024-08-10 21:23:28 +00:00)


In [ ]:
for df, fm in zip(dfsbyfeatmodes, ["rgb","flow","combine"]):
  display(fm)
  display(df)

'rgb'

,AUC,FPR,ACC,BACC,PREC,REC,F1
c1-c2,"99.57 ± 0.0 (99.57,99.57)","1.0 ± 0.0 (1.0,1.0)","98.63 ± 0.0 (98.63,98.63)","92.59 ± 0.0 (92.59,92.59)","71.99 ± 0.0 (71.99,71.99)","86.19 ± 0.0 (86.19,86.19)","78.45 ± 0.0 (78.45,78.45)"
Mean,"99.57 ± 0.0 (99.57,99.57)","1.0 ± 0.0 (1.0,1.0)","98.63 ± 0.0 (98.63,98.63)","92.59 ± 0.0 (92.59,92.59)","71.99 ± 0.0 (71.99,71.99)","86.19 ± 0.0 (86.19,86.19)","78.45 ± 0.0 (78.45,78.45)"


'flow'

,AUC,FPR,ACC,BACC,PREC,REC,F1
c1-c2,"99.67 ± 0.0 (99.67,99.67)","0.97 ± 0.0 (0.97,0.97)","98.85 ± 0.0 (98.85,98.85)","95.8 ± 0.0 (95.8,95.8)","74.11 ± 0.0 (74.11,74.11)","92.56 ± 0.0 (92.56,92.56)","82.32 ± 0.0 (82.32,82.32)"
Mean,"99.67 ± 0.0 (99.67,99.67)","0.97 ± 0.0 (0.97,0.97)","98.85 ± 0.0 (98.85,98.85)","95.8 ± 0.0 (95.8,95.8)","74.11 ± 0.0 (74.11,74.11)","92.56 ± 0.0 (92.56,92.56)","82.32 ± 0.0 (82.32,82.32)"


'combine'

,AUC,FPR,ACC,BACC,PREC,REC,F1
c1-c2,"99.69 ± 0.0 (99.69,99.69)","1.0 ± 0.0 (1.0,1.0)","98.83 ± 0.0 (98.83,98.83)","96.04 ± 0.0 (96.04,96.04)","73.65 ± 0.0 (73.65,73.65)","93.07 ± 0.0 (93.07,93.07)","82.23 ± 0.0 (82.23,82.23)"
Mean,"99.69 ± 0.0 (99.69,99.69)","1.0 ± 0.0 (1.0,1.0)","98.83 ± 0.0 (98.83,98.83)","96.04 ± 0.0 (96.04,96.04)","73.65 ± 0.0 (73.65,73.65)","93.07 ± 0.0 (93.07,93.07)","82.23 ± 0.0 (82.23,82.23)"


time: 82.3 ms (started: 2024-08-10 21:26:28 +00:00)


In [ ]:
df_results = compute_performance_metrics(y_testr, y_probsr, y_predictedr)
df_results.index = [view]
display(df_results)

,AUC,FPR,F1
003,92.6,4.24,67.41


time: 157 ms (started: 2024-07-26 00:23:28 +00:00)
